### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [113]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
#type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"


response = requests.get(query_url)
print(response.json())


{'status': 'OK', 'copyright': 'Copyright (c) 2023 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qua

In [114]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    page_query_url = f"{query_url}&page={page}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url)
    data = response.json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        docs = data["response"]["docs"]
        reviews_list.extend(docs)

        # Print the page that was just retrieved
        print(f"Retrieved page {page}")

    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results for page {page}")
        break

# Optional: Check the first few items in your list
print(json.dumps(reviews_list[:5], indent=4))


Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "su

In [115]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [116]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
print(reviews_df["keywords"].head())


0    [{'name': 'subject', 'value': 'Movies', 'rank'...
1    [{'name': 'subject', 'value': 'Movies', 'rank'...
2    [{'name': 'subject', 'value': 'Movies', 'rank'...
3    [{'name': 'subject', 'value': 'Movies', 'rank'...
4    [{'name': 'subject', 'value': 'Movies', 'rank'...
Name: keywords, dtype: object


In [117]:
# Define the lambda function for extracting the title
extract_title = lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]

# Apply the lambda function to the "headline.main" column
reviews_df["title"] = reviews_df["headline.main"].apply(extract_title)

# Fallback for NaN values in the title column using print_headline
reviews_df["title"] = reviews_df["title"].fillna(reviews_df["headline.print_headline"])

# Optionally, print the DataFrame to check the extracted titles
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [118]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if not isinstance(keyword_list, list):
        return ""  # Return an empty string if the input is not a list

    extracted_keywords = ""
    for item in keyword_list:
        if isinstance(item, dict) and 'name' in item and 'value' in item:
            keyword = f"{item['name']}: {item['value']};"
            extracted_keywords += keyword
        else:
            # Handle cases where item is not a dictionary as expected
            # Adjust this part based on the actual data structure of your items
            extracted_keywords += str(item) + ";"

    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
print(reviews_df["keywords"].head())



0    subject: Movies;creative_works: The Attachment...
1    subject: Movies;persons: Kapur, Shekhar;person...
2    subject: Movies;creative_works: You Can Live F...
3    subject: Movies;creative_works: A Tourist's Gu...
4    subject: Movies;persons: Zlotowski, Rebecca;cr...
Name: keywords, dtype: object


In [119]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles = reviews_df["title"].to_list()
print(movie_titles)


['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [120]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [121]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


# Loop through the titles
for title in movie_titles:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping...")
        time.sleep(1)


    # Add 1 to the request counter
    request_counter += 1

    # URL encode the title for the GET request
    encoded_title = requests.utils.quote(str(title))

    
    # Perform a "GET" request for The Movie Database
    # Execute "GET" request with url
    response = requests.get(url + encoded_title + tmdb_key_string)


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:

        # Get movie id
        movie_id = response.json()["results"][0]["id"]


        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"


        # Execute "GET" request with url
        movie_details_response = requests.get(movie_details_url)
        movie_details = movie_details_response.json()

        
        # Extract relevant data into a dictionary and append it to the tmdb_movies_list list
        movie_dict = {
            "title": movie_details["title"],
            "genres": [genre['name'] for genre in movie_details.get("genres", [])],
            "spoken_languages": [lang['english_name'] for lang in movie_details.get("spoken_languages", [])],
            "production_countries": [country['name'] for country in movie_details.get("production_countries", [])]
        }
        tmdb_movies_list.append(movie_dict)

        
        # Print out the title that was found
        print(f"Found: {movie_details_response.json()['title']}")
        
    except (KeyError, IndexError):
        # Print out the title that was not found
        print(f"Could not find: {title}")
    except json.JSONDecodeError:
        print(f"JSON decoding failed for {title}")
        continue
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {title}: {e}")
        continue



Found: The Attachment Diaries
Could not find: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, A Love Story
Found: In from the Side
Found: After Love
Found: Alcarràs
Found: Nelly & Nadine
Found: Lady Chatterley's Lover
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: The Art of Love
Found: 2099: The Soldier Protocol
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Love & Gelato
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Found: See You Then
Found: La M

In [122]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "French",
            "English"
        ],
        "production_countries": [
            "Canada"
        ]
    },
    {
        "title": "A Tourist's Guide to Love",
        "genres": [
            "Romance",
            "Comedy",
            "Adventure"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Other People's Children",
        "genres": [
            "Drama"
        ],
       

In [123]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()


,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,"[Drama, Romance]","[French, English]",[Canada]
2,A Tourist's Guide to Love,"[Romance, Comedy, Adventure]",[English],[United States of America]
3,Other People's Children,[Drama],[French],[France]
4,One True Loves,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"


### Merge and Clean the Data for Export

In [138]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title")
merged_df.head(20)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"[Drama, Romance]","[French, English]",[Canada]
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,[Drama],[English],[United States of America]
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine,"[Romance, Comedy]",[English],[United States of America]
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Love in the Time of Fentanyl,[Documentary],[English],"[Canada, United States of America]"
7,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None,After Love,[Drama],"[English, Arabic, French, Urdu]",[United Kingdom]
8,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil.

In [139]:
# Remove list brackets and quotation marks on the columns containing lists
# List of columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Characters to remove
characters_to_remove = ["[", "]", "'"]

for column in columns_to_fix:
    # Check if the column exists in the DataFrame
    if column in merged_df.columns:
        # Convert the column to type 'str'
        merged_df[column] = merged_df[column].astype(str)

        # Loop through characters to remove
        for char in characters_to_remove:
            merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
merged_df.head(30)



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"Drama, Romance","French, English",Canada
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"Romance, Drama, Comedy",English,"Germany, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,Drama,English,United States of America
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine,"Romance, Comedy",English,United States of America
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Love in the Time of Fentanyl,Documentary,English,"Canada, United States of America"
7,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None,After Love,Drama,"English, Arabic, French, Urdu",United Kingdom
8,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"subject: Movies;persons:

In [140]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])


In [141]:
# Delete duplicate rows
merged_df = merged_df.drop_duplicates()

# Reset the index
merged_df = merged_df.reset_index(drop=True)

# Display the updated DataFrame
merged_df.head(20)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,"Drama, Romance","French, English",Canada
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None,One True Loves,"Romance, Drama, Comedy",English,"Germany, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None,A Thousand and One,Drama,English,United States of America
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None,Your Place or Mine,"Romance, Comedy",English,United States of America
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None,Love in the Time of Fentanyl,Documentary,English,"Canada, United States of America"
7,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,None,After Love,Drama,"English, Arabic, French, Urdu",United Kingdom
8,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"subject: Movies;persons: Simon, Carla;creative...",2023-01-05T12:00:03+0000,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,None,Alcarràs,Drama,Catalan,Spain
9,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,subject: Documentary Films and Programs;person...,2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None

In [142]:
# Define the file path where you want to save the CSV file
file_path = './output/movies_data_output.csv'  # Replace with your desired file path

# Export the DataFrame to a CSV file without the index
merged_df.to_csv(file_path, index=False)

print("Data exported to CSV successfully.")


Data exported to CSV successfully.
